# 7.3 모델의 성능을 최대로 끌어올리기

### 7.3.1 고급 구조 패턴

#### 배치 정규화

데이터에서 평균을 빼서 데이터를 원점에 맞추고 표준 편차로 나누어 데이터의 분산을 1로 만듭니다.

```python
normalized_data = (data - np.mean(data, axis=...)) / np.std(data, axis=...)
```

훈련하는 동안 평균과 분산이 바뀌더라도 이에 적응하여 데이터를 정규화합니다. 훈련 과정에 사용된 배치 데이터의 평균과 분산에 대한 지수 이동 평균(exponential moving average)을 내부에 유지합니다. 배치 정규화의 주요 효과는 잔차 연결과 매우 흡사하게 그래디언트의 전파를 도와주는 것입니다. 결국 더 깊은 네트워크를 구성할 수 있습니다. 매우 깊은 네트워크라면 여러 개의 BatchNormalization 층을 포함해야 훈련할 수 있습니다.

```python
conv_model.add(layers.Conv2D(32, 3, activation='relu'))  # Conv2D 층 다음에
conv_model.add(layers.Batchnormalization())

dense_model.add(layers.Dense(32, activation='relu'))  # Dense 층 다음에
dense_model.add(layers.BatchNormalization())
```

#### 깊이별 분리 합성곱

Conv2D를 대체하면서 더 가볍고(훈련할 모델 파라미터가 더 적고) 더 빨라(부동 소수 연산이 더 적고) 모델의 성능을 몇 퍼센트 포인트 높일 수 있는 층이 있다면 어떨까요? 이것이 깊이별 분리 합성곱(depthwise separable convolution) 층이 하는 일입니다(SeparableConv2D).

```python
from keras.models import Sequential, Model
from keras import layers

height = 64
width = 64
channels = 3
num_classes = 10

model = Sequential()
model.add(layers.SeparableConv2D(32, 3, activation='relu', input_shape=(height, width, channels,)))
model.add(layers.SeparableConv2D(64, 3, activation='relu'))
model.add(layers.MaxPooling2D(2))

model.add(layers.SeparableConv2D(64, 3, activation='relu'))
model.add(layers.SeparableConv2D(128, 3, activation='relu'))
model.add(layers.MaxPooling2D(2))

model.add(layers.SeparableConv2D(64, 3, activation='relu'))
model.add(layers.SeparableConv2D(128, 3, activation='relu'))
model.add(layers.GlobalAveragePooling2D())

model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(num_classes, activation='softmax'))

model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
```

### 7.3.2 하이퍼파라미터 최적화

딥러닝 모델을 만들 때 무작위로 보이는 결정을 많이 해야합니다. 얼마나 많은 층을 쌓아야 할까요? 층마다 얼마나 많은 유닛이나 필터를 두어야 할까요? relu 활성화 함수를 사용해야 할까요? 아니면 다른 함수를 사용해야 할까요? 어떤 층 뒤에 BatchNormalization을 사용해야 할까요? 드롭아웃은 얼마나 해야할까요?

하루종일 하이퍼파라미터를 수정하는 것은 사람이 할 일은 아닙니다. 기계에 위임하는 것이 더 낫습니다.

전형적인 하이퍼파라미터 최적화 과정
1. 일련의 하이퍼파라미터를 (자동으로) 선택합니다.
2. 선택된 하이퍼파라미터로 모델을 만듭니다.
3. 훈련 데이터에 학습하고 검증 데이터에서 최종 성능을 측정합니다.
4. 다음으로 시도할 하이퍼파라미터를 (자동으로) 선택합니다.
5. 이 과정을 반복합니다.
6. 마지막으로 테스트 데이터에서 성능을 측정합니다.

여러가지 기법을 사용할 수 있습늬다. 베이지만 최적화(bayesian optimization), 유전 알고리즘(genetic algorithms), 간단한 랜덤 탐색(random search) 등 입니다.

이 분야가 어렵고 아직 초창기이기 때문에 모델 최적화에 사용할 도구가 매우 적습니다. 랜덤 탐색보다 더 나은 도구는 하이퍼파라미터 최적화를 위한 파이썬 라이브러리인 Hyperopt입니다. 다른 라이브러로리ㅗ Hyperas는 Hyperopt와 연동하여 케라스 모델에 사용할 수 있습니다.

### 7.3.3 모델 앙상블

앙상블은 여러 개의 다른 모델의 예측을 합쳐서 더 좋은 예측을 만듭니다.

```python
# 4개의 다른 모델을 사용하여 초기 예측을 계산합니다.
preds_a = model_a.predict(x_val)
preds_b = model_b.predict(x_val)
preds_c = model_c.predict(x_val)
preds_d = model_d.predict(x_val)

final_preds = 0.25 * (preds_a + preds_b + preds_c + preds_d)  # 새로운 예측은 어떤 초기 예측보다 더 정확해야 합니다.
```

또는

```python
preds_a = model_a.predict(x_val)
preds_b = model_b.predict(x_val)
preds_c = model_c.predict(x_val)
preds_d = model_d.predict(x_val)

final_preds = 0.5 * preds_a + 0.25 * preds_b + 0.1 * preds_c + 0.15 * preds_d  # 가중치 (0.5, 0.25, 0.1, 0.15)는 경험적으로 학습되었다고 가정합니다.
```

다양성이 앙상블의 힘입니다.

이런 이유 때문에 가능한 최대한 다르면서 좋은 모델을 앙상블해야 합니다.

최근에 실전에서 매우 성공적으로 사용되는 기본 앙상블 스타일은 딥러닝과 얕은 모델을 섞은 넓고 깊은(wide and deep) 모델입니다.

### 7.3.4 정리
- 고성능 심층 컨브넷을 만들려면 잔차 연결, 배치 정규화, 깊이별 분리 합성곱을 사용해야 합니다.
- 십층 네트워크를 만들 때 많은 하이퍼파라미터와 네트워크 구조를 선택해야 합니다.
- 머신 러닝 경연대회에서 우승하거나 어떤 문제에서 최상의 결과를 얻으려면 대규모로 모델을 앙상블하게 됩니다.